# QR Constraint Networks 101

This notebook provides a brief example of how a <b>Constraint Network Object</b> can be...
* represented in JSON or Python dictionary formats
* instantiated from a JSON file or Python dictionary
* serialized to a JSON file/string or Python dictionary
* summarized
* propagated
* queried for details about node and edge attributes

Two different types of contraint algebras are used here:
1. The spatial constraint algebra, [Region Connection Calculus 8 (RCC8)](https://en.wikipedia.org/wiki/Region_connection_calculus)
1. The temporal interval & point algebra defined by Reich in ["Intervals, Points, and Branching Time", 1994](https://www.researchgate.net/publication/220810644_Intervals_Points_and_Branching_Time)

## Imports

In [1]:
import qualreas as qr
import os

## Path to Network & Algebra

To begin, we will instantiate a Constraint Network and it's corresponding Algebra from JSON files.

Each are kept in separate directories, 'Networks' and 'Algebras', within a top-level 'qualreas' directory, with the full path defined here using an environment variable:

In [2]:
qr_path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

Once defined, an Algebra's JSON format should remain unchanged. The name of the Algebra used by a Network can then be stored in the Network's definition (in JSON) regardless of where the Network's JSON file resides.  So, we only need provide the path to the directory containing Algebra files:

In [3]:
alg_dir = os.path.join(qr_path, "Algebras")

Networks, on the other hand, could be numerous and change often.  So, we need to provide the full path to the Network's JSON file.

In [4]:
rcc8_file = os.path.join(qr_path, "Networks", "rcc8_example.json")

## Constraint Network in JSON Format

Here's what a network looks like in JSON format.

A node is represented as a list of two things:
* Node ID
* List of ontology classes the node belongs to

An edge is represented as a list of three things, representing a directed edge, labeled with a constraint:
* Head Node ID
* Target Node ID
* Constraint

Two Things to Note:
1. The Wikipedia page on RCC8 represents disjunctions of constraints as sets, e.g., {DC, EC}, whereas QR represents this with the string "DC|EC".  Constraint sets represent disjunctions of relation statements, e.g., (Head DC|EC Tail) <==> (Head DC Tail) OR (Head EC Tail).
1. Constraints can be abbreviated using a dictionary of abbreviations.  For example, below, the constraint "DC|EC" is abbreviated as "dec".  By the way, internally, the QR module stores and operates on constraint sets as <i>bitsets</i>.
1. No constraints are given for the Road-to-Property1 or Road-to-Property2 edges.  The meaning then is that all RCC8 relations are possible for those two edges.  This can be seen in the first summary of the network farther below.

In [5]:
!cat {rcc8_file}

{
    "name": "RCC8 Example 1",
    "algebra": "RCC8_Algebra",
    "abbreviations": {"dec": "DC|EC"},
    "description": "See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples",
    "nodes": [
        ["House1", ["Region"]],
        ["House2", ["Region"]],
        ["Property1", ["Region"]],
        ["Property2", ["Region"]],
        ["Road", ["Region"]]
    ],
    "edges": [
        ["House1", "House2", "DC"],
        ["House1", "Property1", "TPP|NTPP"],
        ["House1", "Property2", "dec"],
        ["House1", "Road", "EC"],
        ["House2", "Property1", "dec"],
        ["House2", "Property2", "NTPP"],
        ["House2", "Road", "EC"],
        ["Property1", "Property2", "dec"],
        ["Road", "Property1"],
        ["Road", "Property2"]
    ]
}

## Instantiate the Constraint Network Object

In [6]:
rcc8_net = qr.Network(algebra_path=alg_dir, json_file_name=rcc8_file)

print(rcc8_net)

<Network--RCC8 Example 1--RCC8_Algebra>


## Summarize the Network

This network is an example on the Wikipedia page for RCC8 (see "description" field, above).

Below is a summary of the Network Object just instantiated.

The format is:
* Network Name: Number of Nodes, Number of Edges
* Algebra Name
* Head Node ID 1:
    * => Tail Node ID 1: Constraint 1
    * => Tail Node ID 2: Constraint 2
    * ...
* and so on ...

In [7]:
rcc8_net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC|EC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC|EC
    => Property2: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
    => Property2: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI


## Perform Constraint Propagation

After propagation, note the change in the constraints between the Road and the two Properties.

In [8]:
rcc8_net.propagate()
rcc8_net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP


## Get Entity by Name

<b>get_entity_by_name</b> returns an object. Use the object's methods to access it's attributes.

In [9]:
entity = rcc8_net.get_entity_by_name("House1")
entity

SpatialObject(['Region'] 'House1')

In [10]:
entity.name

'House1'

In [11]:
entity.classes

['Region']

## Get Edge by Head & Tail Names

<b>get_edge_by_names</b> returns a tuple of three things: (Head Node ID, Tail Node ID, Constraint)

In [12]:
edge = rcc8_net.get_edge_by_names("Property1", "Road")
edge

('Property1', 'Road', 'EC|PO')

In [13]:
edge[2]

'EC|PO'

## Network to Dictionary

Note: The only differences between JSON and the dictionary output by <b>to_dict</b> are the single quotes instead of double quotes required by JSON.

In [14]:
rcc8_net_dict = rcc8_net.to_dict()

rcc8_net_dict

{'name': 'RCC8 Example 1',
 'algebra': 'RCC8_Algebra',
 'description': 'See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples',
 'nodes': [['House1', ['Region']],
  ['House2', ['Region']],
  ['Property1', ['Region']],
  ['Property2', ['Region']],
  ['Road', ['Region']]],
 'edges': [['House1', 'House2', 'DC'],
  ['House1', 'Property1', 'NTPP|TPP'],
  ['House1', 'Property2', 'DC|EC'],
  ['House1', 'Road', 'EC'],
  ['House2', 'Property1', 'DC'],
  ['House2', 'Property2', 'NTPP'],
  ['House2', 'Road', 'EC'],
  ['Property1', 'Property2', 'DC|EC'],
  ['Property1', 'Road', 'EC|PO'],
  ['Property2', 'Road', 'PO|TPPI']]}

## Dictionary to Network

Instantiating a Network from a dictionary is similar to using the JSON format.  Although it is not shown below, we can define and use abbreviations for constraints, and we can leave the constraints off of edge definitions to indicate that all relations are possible.

In [15]:
rcc8_net_from_dict = qr.Network(algebra_path=alg_dir, network_dict=rcc8_net_dict)

print(rcc8_net_from_dict)

rcc8_net_from_dict.summary()

<Network--RCC8 Example 1--RCC8_Algebra>

RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP


## Network to JSON

A simple way to serialize a network in JSON format is to first convert it to a dictionary using <b>to_dict</b> and then use <b>json.dump()</b> or <b>json.dumps()</b> to write it out to a file or convert it to a string, respectively.

However, either way, the resulting file or string are not pretty printed.

#### Network to JSON File

In [16]:
import json

rcc8_json_file = os.path.join(qr_path, "Networks", "rcc8_test1.json")

with open(rcc8_json_file, "w") as fout:
    json.dump(rcc8_net_dict, fout)

In [17]:
!cat {rcc8_json_file}

{"name": "RCC8 Example 1", "algebra": "RCC8_Algebra", "description": "See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples", "nodes": [["House1", ["Region"]], ["House2", ["Region"]], ["Property1", ["Region"]], ["Property2", ["Region"]], ["Road", ["Region"]]], "edges": [["House1", "House2", "DC"], ["House1", "Property1", "NTPP|TPP"], ["House1", "Property2", "DC|EC"], ["House1", "Road", "EC"], ["House2", "Property1", "DC"], ["House2", "Property2", "NTPP"], ["House2", "Road", "EC"], ["Property1", "Property2", "DC|EC"], ["Property1", "Road", "EC|PO"], ["Property2", "Road", "PO|TPPI"]]}

#### Network to JSON String

In [18]:
json.dumps(rcc8_net_dict)

'{"name": "RCC8 Example 1", "algebra": "RCC8_Algebra", "description": "See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples", "nodes": [["House1", ["Region"]], ["House2", ["Region"]], ["Property1", ["Region"]], ["Property2", ["Region"]], ["Road", ["Region"]]], "edges": [["House1", "House2", "DC"], ["House1", "Property1", "NTPP|TPP"], ["House1", "Property2", "DC|EC"], ["House1", "Road", "EC"], ["House2", "Property1", "DC"], ["House2", "Property2", "NTPP"], ["House2", "Road", "EC"], ["Property1", "Property2", "DC|EC"], ["Property1", "Road", "EC|PO"], ["Property2", "Road", "PO|TPPI"]]}'

## An Example of Temporal Reasoning

Here we'll use the temporal interval & point algebra, <b>Extended_Linear_Interval_Algebra</b>, defined by Reich in ["Intervals, Points, and Branching Time", 1994](https://www.researchgate.net/publication/220810644_Intervals_Points_and_Branching_Time)

This algebra extends Allen's algebra of proper time intervals to include time points.

The relation, "PF", is "PointFinishes", "PS" is "PointStarts", and "PE" is "PointEquals".  "PFI" & "PSI" are the converses of "PF" and "PS", resp.

For example,
* <i>MondayMidnight PF Monday</i>
* <i>MondayMidnight PS Tuesday</i>
* <i>MondayMidnight PE MondayMidnight</i>

In [19]:
time_net_dict = {
    'name': 'Simple Temporal Constraint Network',
    'algebra': 'Extended_Linear_Interval_Algebra',
    'description': 'A simple example of a Network of Time Points integrated with Time Intervals',
    'nodes': [
        ['Monday', ['ProperInterval']],
        ['MondayMidnight', ['Point']],
        ['Tuesday', ['ProperInterval']],
        ['MondayPM', ['ProperInterval']]
    ],
    'edges': [
        ['Monday', 'Tuesday', 'M'],
        ['MondayMidnight', 'Monday', 'PF'],
        ['MondayPM', 'Monday', 'F']
    ]
}

In [20]:
time_net = qr.Network(algebra_path=alg_dir, network_dict=time_net_dict)

print(time_net.description)

time_net.summary()

A simple example of a Network of Time Points integrated with Time Intervals

Simple Temporal Constraint Network: 4 nodes, 10 edges
  Algebra: Extended_Linear_Interval_Algebra
  Monday:
    => Monday: E
    => Tuesday: M
    => MondayMidnight: PFI
    => MondayPM: FI
  Tuesday:
    => Tuesday: E
    => Monday: MI
  MondayMidnight:
    => MondayMidnight: PE
    => Monday: PF
  MondayPM:
    => MondayPM: E
    => Monday: F


In [21]:
time_net.propagate()
time_net.summary()


Simple Temporal Constraint Network: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  Monday:
    => Monday: E
    => Tuesday: M
    => MondayMidnight: PFI
    => MondayPM: FI
  Tuesday:
    => Tuesday: E
    => Monday: MI
    => MondayMidnight: PSI
    => MondayPM: MI
  MondayMidnight:
    => MondayMidnight: PE
    => Monday: PF
    => Tuesday: PS
    => MondayPM: PF
  MondayPM:
    => MondayPM: E
    => Monday: F
    => Tuesday: M
    => MondayMidnight: PFI


Note that with regard to the point, <i>MondayMidnight</i> as defined in <b>time_net_dict</b>, above, we only specified:

><i>MondayMidnight PF Monday</i>

That is, <i>MondayMidnight</i> is the end point (<i>PointFinishes</i>) of the interval, <i>Monday</i>.

After propagation, we've inferred that:

><i>MondayMidnight PS Tuesday</i>

That is, <i>MondayMidnight</i> is the begin point (<i>PointStarts</i>) of the interval, <i>Tuesday</i>.

And we've inferred the corresponding converses of these statements (i.e., using the relations <i>PFI</i> and <i>PSI</i>).

## A Final Note

The Network object in QR is a subclass of [networkx.digraph](https://networkx.github.io/documentation/stable/reference/classes/digraph.html), which has functionality for loading/saving from/to JSON format. But none of the JSON load/save functionality in networkx supported the needs of QR networks.  For example:
* <b>to_dict_of_dicts</b>: Keeps edge attributes but loses the graph and node attributes
* <b>to_dict_of_lists</b>: Loses all attributes
* <b>json_graph.node_link_data</b>: Retains graph & link attributes, but loses node attributes
* <b>nx.readwrite.json_graph.node_link_graph</b>: Doesn't work for DiGraphs
* GraphML keeps graph and edge attributes, but loses node attributes
* ...and so on...

So, the bespoke JSON format, described in this notebook, was developed for QR.